<a href="https://colab.research.google.com/github/XueTong1225/hipims_tutorial/blob/main/tutorial_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcom to HiPIMS!


This document gives you a guidance and embeds the code to run the hipims code for case study.
You can run this tutorial in a couple of ways:

**In the cloud:** This is an easy way to get started! Use link to opens an integrated notebook in Google Colab.

**Locally:** This option requires you to setup HiPIMS first on your local machine. Download the notebook or copy the code into your favorite IDE. We recommand you using CUDA 12.4 and python>=3.10.

## About
HiPIMS names for High-Performance Integrated hydrodynamic Modelling System. It uses state-of-art numerical schemes (Godunov-type finite volume) to solve the 2D shallow water equations for flood simulations. To support high-resolution flood simulations, HiPIMS is implemented on multiple GPUs (Graphics Processing Unit) using CUDA/C++ languages to achieve high-performance computing.

**Package Author:** Xue Tong, Jiaheng Zhao

**Guidance and leadership:** Prof. Qiuhua Liang

**Affiliation:** Loughborough University


## 1. Install and import the package

### 1.1 Recommended applications:
- [**Visual Studio Code**](https://code.visualstudio.com/download) as Python IDE
    - Extensions: Python, jupyter, Remote - SSH


### 1.2 Select runtime type
- Hardware accelerator: T4 GPU
- Runtime version: 2025.07

### 1.3 Install HiPIMS and nessary packages in Terminal:

In [ ]:
pip install hipims

if ```pip install hipims``` not work, try ```python -m pip install hipims``` instead.

### 1.4 Test HiPIMS

In [ ]:
from hipims.InputHipims import InputHipims
from hipims import flood
from hipims import visualization as visual

## 2. Data Formats and Types
To run the flood model, you need to prepare specific data types. Below are the minimum required data and additional **data types** that can enhance your simulations.

1. **Spatial Distribution Data (SDD)**:  
   These data describe spatial patterns and variations across different locations. The supported formats include:  
   - **Single Value**: Represents a uniform value across the entire spatial domain, used when the phenomenon is consistent across all locations.
   - **`np.array` (NumPy Array)**: Allows users to input customised values as an array to represent spatial data.  
     *Note: The shape of the `np.array` must match the shape of the DEM to maintain spatial consistency.*
   - **`tif` (GeoTIFF) File**: A raster data format that includes geographic coordinate information, allowing for detailed spatial modelling.  
     *Note: The grid size of the GeoTIFF file must match that of the DEM.*

2. **Time Series Data (TSD)**:  
   These data represent temporal dynamics and support the following formats:  
   - **Single Value**: Represents a uniform value across the entire time period, implying no variation over time.
   - **`np.array` (NumPy Array)**: Allows users to input customised time series data. The first column represents the time series (unit: seconds), and the subsequent columns contain the corresponding values.
   - **File**: Time series data can be stored in a file, such as a `.txt` or `Excel file`.  
     *Note: If the data are on a specific sheet in an Excel file, use the format `['filename', 'sheetname']`. Otherwise, use `'filename'`.*

## 3. Running HiPIMS for Catchment Case Study
3.1 & 3.9 are necessary.

3.2-3.8 are optional, call only differ from default.

*Note: SDD refers to Spatial Distribution Data; TSD refers to Time Series Data.*


### 3.1 Quickstart

To define the model case, use the following:

- **Data Requirements**:
  - `DEM.tif` : Digital Elevation Model (DEM), providing terrain elevation data necessary for calculating water flow paths.

- **Function**: `InputHipims`

  **Keyword Arguments**:

<div align="center">

| Argument           | Type    | Description                                                               |
|--------------------|---------|---------------------------------------------------------------------------|
| `input_folder`      | String  | Path to the folder containing the case input files.                              |
| `output_folder`    | String  | Path to the folder where the simulation output files will be saved.        |
| `dem`         | String  | Path to the Digital Elevation Model (DEM) file, providing terrain data.    |

</div>


In [ ]:
pip show hipims

In [ ]:
# Minimal requirements
# Define simulation domain
input_folder = r"/usr/local/lib/python3.11/dist-packages/hipims/sample/input"
output_folder = r"/usr/local/lib/python3.11/dist-packages/output"
dem = r"DEM.tif"

# for Linux
# input_folder = 'C:\HiPIMS_2024\src\hipims\sample\input'
# output_folder = 'C:\HiPIMS_2024\src\hipims\sample\output'
# dem = 'DEM.tif'

inputs = InputHipims(input_folder = input_folder,
                     output_folder= output_folder,
                     dem = dem)

Want to see your simulation area? Run:

In [ ]:
import os
dem_path = os.path.join(input_folder, dem)
outline_path = os.path.join(input_folder, 'outline.shp')
visual.domain(file_path = dem_path, outline=outline_path)

### 3.2 Set Rainfall
Define the rainfall data for the simulation. By default, there is no rainfall.

- **Function**: `set_rainfall`

  **Keyword Arguments:**

<div align="center">

| Argument      | Type                             | Unit                     | Default | Description                                                                                           |
|---------------|----------------------------------|--------------------------|---------|-------------------------------------------------------------------------------------------------------|
| `rain_mask`   | SDD   | N/A     | `0`     | A mask that defines the rainfall station indices. indices.                                                     |
| `rain_source` | TSD   | mm/hr   | `0`     | The source of rainfall data. The first column represents time, and the subsequent columns correspond to the indices in `rain_mask`, starting from 0. |

</div>


In [ ]:
inputs.set_rainfall(rain_mask='rainMask.tif',rain_source='rainfall_source.txt')

Want to see the rainfall mask and time series? Run:

In [ ]:
# show the rainfall mask
rain_path = os.path.join(input_folder, 'rainMask.tif')
visual.rain_mask(file_path = rain_path,
                 outline = outline_path)

# show the raifnall time series
from datetime import datetime
rain_source_path = os.path.join(input_folder, 'rainfall_source.txt')
start_date = datetime(2015,12,4,17,0,0)
visual.rainfall_time_series(file_path = rain_source_path,
                            start_date = start_date)

### 3.3 Set Up Model Parameters

Define the model parameters to control the simulation's configuration.

- **Function**: `set_model_parameters`

  **Keyword Arguments**:

<div align="center">

| Argument              | Type     | Default  | Description                                                            |
|-----------------------|----------|----------|------------------------------------------------------------------------|
| `num_GPU`             | Integer  | `1`      | Number of GPU devices to use.                                          |
| `start_time`          | Numeric  | `0`      | Simulation start time for the event in seconds.                        |
| `GPU_device_ID`       | Integer  | `0`      | ID of the GPU device.                                                  |
| `end_time`            | Numeric  | `3600`   | Total simulation time in seconds.                                      |
| `output_interval`     | Numeric  | `600`    | Specifies how frequently (in seconds) the output files are saved.      |
| `second_order`        | Boolean  | `False`  | Specifies whether to use a second-order solver.                        |
| `projected_coordinate`| Boolean  | `False`  | Indicates whether the geographic coordinate is in a projected coordinate system.                        |

</div>

For example, if the case starts from 01-Jan-2024 00:00:00 and ends at 01-Jan-2024 00:03:00 and you would like to see the results for every 10 minuts, then the star_time should be `0`, the output_intervel is `600` = 10 min * 60 s/min = 600 s, and the end_time is `10800` = 3 hour * 3600 s/hour = 10800 s.

In [ ]:
inputs.set_model_parameters(output_interval=3600, end_time=3600)

### 3.4 Set Initial Condition

Define the initial conditions for the simulation. By default, the simulation area is considered completely dry.
- **Function**: `set_initial_condition`

  **Keyword Arguments:**

<div align="center">

| Argument     | Type  | Unit       | Default | Description                                |
|--------------|-------|------------|---------|--------------------------------------------|
| `h0`         | SPD   | Metres (m) | `0`     | Initial water depth.                       |
| `hU0x`       | SPD   | Metres (m) | `0`     | Initial unit discharge in the x-direction. |
| `hU0`        | SPD   | Metres (m) | `0`     | Initial unit discharge in the y-direction. |

</div>



In [ ]:
inputs.set_initial_condition(h0='depth.tif',
                              hU0x='hux.tif',
                              hU0y='huy.tif')

Want to see initial condition? Run:

### 3.5. Set Boundary Condition

Define the Input/Output (IO) boundaries at the edges of the model domain. These boundaries typically represent river inflow from upstream or river water depth/tidal levels downstream.

- **Function**: `set_boundary_condition`

  **Keyword Arguments:**

| Argument           | Type                   | Description                                                                                                    |
|--------------------|------------------------|----------------------------------------------------------------------------------------------------------------|
| `bound_list`       | List of dictionaries   | A list where each dictionary represents a boundary condition for the model.                                    |
| - `extent`         | Array                  | Specifies the boundary's spatial extent using minimum and maximum X and Y coordinates in the format `[x_min, x_max, y_min, y_max]`. |
| - `type`           | String                 | Defines the type of boundary condition (see **Boundary Condition Types** below for details).                   |
| - `source`         | TSD / None             | Specifies the source of data for the boundary condition. (see **Boundary Condition Types** below for details). |
| `outline_boundary` | String                 | Specifies the condition applied to the model's outer boundary.                                                 |

- **Boundary Condition Types**

  The following types of boundary conditions are available:

<div align="center">

| Boundary Type   | Description                                                                 | Source | Unit                        |
|-----------------|-----------------------------------------------------------------------------|--------------------------------------|------------------------------|
| `RIGID`         | A solid, non-movable boundary.                                               | None | N/A                          |
| `WALL_SLIP`     | Allows slip conditions along a wall boundary.                                | None | N/A                          |
| `OPEN`          | An open boundary where flow can freely enter or exit.                        | None  | N/A                          |
| `H_GIVEN`       | A boundary with a specified water depth.                                     | TSD          | m                |
| `Q_GIVEN`       | A boundary with a specified flow discharge.                                  | TSD| m³/s|
| `WL_GIVEN`      | A boundary with a specified water level.                                     | TSD| m               |
| `FALL`          | A specific condition, such as a falling water level.                         | None           | N/A |

</div>

In [ ]:
boundary1 = {'extent': [339490.05, 339516.74, 552724.15, 552731.72], #[left, right, bottom, top]
             'type': 'WL_GIVEN',
             'source': 23.8}
boundary2 = {'extent': [343488.09, 343505.57, 551295.62, 551320.47],
             'type': 'Q_GIVEN',
             'source': ['Q_source.xlsx', 'Petteril']}
boundary3 = {'extent': [342871.50, 342931.40, 557885.14, 557918.513],
             'type': 'Q_GIVEN',
             'source': ['Q_source.xlsx', 'Eden']}
boundary4 = {'extent':  [337093.251, 337163.418, 557213.469, 557293.638],
             'type': 'OPEN',
             'source': None}

bound_list = [boundary1, boundary2, boundary3, boundary4 ]

inputs.set_boundary_condition(bound_list = bound_list,
                              outline_boundary = 'FALL')

Show your boundary? Run:

In [ ]:
visual.domain(file_path = dem_path, bound_list= bound_list, outline=outline_path)

### 3.6 Land Cover and Related Processes
Land cover types, such as forests, urban areas, or fields, affect the speed and volume of surface water flow by influencing processes like surface friction, infiltration, and sewer drainage.

#### 3.6.1 Defining Land Cover
Define the land cover data for the simulation to represent different types of land use within the case domain.

- **Function**: `set_landuse`

    **Keyword Argument:**

<div align="center">

| Argument        | Type                             | Default | Description                                                |
|-----------------|----------------------------------|---------|------------------------------------------------------------|
| `landuse_mask`  | SPD (Spatial Distribution Data)  | `0`     | Indexes the land use distribution within the case domain.   |

</div>

In [ ]:
inputs.set_landuse(landuse_mask='Landcover.tif')

To show the Landuse data, run:

In [ ]:
land_path = os.path.join(input_folder, 'Landcover.tif')
visual.land_mask(file_path = land_path,
                 categories={1:'water', 2:'Land'},
                 title = 'Landuse mask',
                 outline = outline_path)

#### 3.6.2 Common Data Formats for Land Cover-Based Parameters
All land cover-based parameters share the following data formats:
- **Single Value**:  A numeric value applied for all land use categories.
- **`np.array` (NumPy Array)**: An array of numeric values corresponding to the land use categories defined in the `landuse_mask`. The values should be ordered from the smallest to the largest land use indices.
- **List**:  A list format that allows for custom settings:
   - **`'default_value'`**: The default parameter value for all land use categories except specified types.
   - **`'special_land_type_value'`**: The index of a specific land use category requiring a different parameter value.
   - **`'special_param_value'`**: The value corresponding to the `special_land_type_value` for the specified parameter.

#### 3.6.3 Friction
Friction in hydrodynamic modelling refers to the resistance water encounters as it flows over different surfaces, such as riverbeds, floodplains, or urban landscapes. This resistance is primarily influenced by the roughness of the surface, which slows down the water and dissipates its energy.

- **Function**: `set_manning`

    **Keyword Argument:**

<div align="center">

| Argument        | Type                             | Default | Description                                                |
|-----------------|----------------------------------|---------|------------------------------------------------------------|
| `manning`  | Land Cover-Based Parameters  | `0`     | The roughness coefficient. Higher values represent rougher surfaces, slowing down water flow.  |

</div>


In [ ]:
inputs.set_manning(manning={'default_value': 0.035,
                            'special_land_type_value': [1],
                            'special_param_value': [0.03]})

#### 3.6.4 Infiltration

Infiltration is the process by which water on the ground surface enters the soil. In HiPIMS, the [**Green-Ampt model**](https://en.wikipedia.org/wiki/Green%E2%80%93Ampt_model) is used to calculate the amount of water that infiltrates into the soil over time.

There are two methods for setting up infiltration.

***!!! Run one of the following methods depending on your own case:***



##### Method 1: Manual Parameter Setting
Directly assign specific values for each parameter based on different land cover types.

- **Function**: `set_infiltration`

**Keyword Arguments:**

<div align="center">

| Argument                 | Type                         | Default | Description                                                                 |
|--------------------------|------------------------------|---------|-----------------------------------------------------------------------------|
| `cumulative_depth`       | Land Cover-Based Parameters  | `0`     | Cumulative infiltration depth since the start of the simulation (m).         |
| `hydraulic_conductivity` | Land Cover-Based Parameters  | `0`     | The rate at which water moves through the soil, depending on soil properties (m/s). |
| `capillary_head`         | Land Cover-Based Parameters  | `0`     | Soil suction at the wetting front, affecting water infiltration (m).         |
| `water_content_diff`     | Land Cover-Based Parameters  | `0`     | Difference in water content between saturated and initial conditions (m³/m³). |

</div>

In [ ]:
inputs.set_infiltration(cumulative_depth=0,
                        hydraulic_conductivity=0,
                        capillary_head=0,
                        water_content_diff=0)


##### Method 2: Automatic Parameter Setting by Soil Type
Automatically determine parameter values based on the soil type associated with each land cover type.

Soil types are based on the USDA Soil Classification: Clay, Silty Clay, Sandy Clay, Clay Loam, Silty Clay Loam, Sandy Clay Loam, Silt Loam, Loam, Sandy Loam, Loamy Sand, Sand, Impervious Surface, Water.

- **Function**: `set_soil_type`

    **Keyword Arguments:**

<div align="center">

| Argument | Format | Description |
|----------|--------|-------------|
| `soil_type`   | List | Specifies the soil type settings for different land use categories.     |
| - `default_soil_type` | String | Default soil type for all land use categories unless specified otherwise. |
| - `special_land_type_value` | list of values / None | Index of a specific land use category requiring a different soil type.|
| - `special_soil_type` | list of string / None | The soil type corresponding to the `special_land_type_value`.|
</div>



In [ ]:
inputs.set_soil_type(soil_type={'default_soil_type': 'Impervious Surface',
                                'special_land_type_value': [1],
                                'special_soil_type': ['Water']})

#### 3.6.5 Sewer System
By defining areas in the model that drain water into the sewer system, users can simulate how water is removed from specific land cover types, such as roads, at a controlled rate.

- **Function**: `set_sewer_sink`

    **Keyword Arguments:**

<div align="center">

| Argument     | Type                        | Default | Description                                                               |
|--------------|-----------------------------|---------|---------------------------------------------------------------------------|
| `sewer_sink` | Land Cover-Based Parameters | `0`     | Specifies the rate at which water is drained into the sewer system maximumly (m/s). |

</div>


In [ ]:
inputs.set_sewer_sink(sewer_rate=0)

### 3.7 Set river gauges
Set gauges with the river channel to monitor timeseries of water level and velocity

In [ ]:
import numpy as np
gauge_coords = np.array([[340194, 556641]])
inputs.set_gauges_position(gauge_coords)

### 3.8 Check your data before running

In [ ]:
print(inputs)

### 3.9 Run flood model

The simulation speed may vary depends on the GPU type, grid cell numbers, and the complexity of the simulation case. In this demonstration, it may take you around 20 minutes on Google Colab.

In [ ]:
flood.run(inputs)

## 4. Visualize outputs

Show the maximum water depth within the simulated period. Run:

In [ ]:
max_depth_path = os.path.join(output_folder, "h_max_600.tif")

visual.inundation_depth(file_path = max_depth_path,
                        title = 'Maximum water depth (m)',
                        basemap=dem_path,
                        outline=outline_path)

## 5. Use outputs for flood risk assessment

There are lots of method to assess flood risk using HiPIMS outputs. Here we provide an easy demonstration, but you can use your own methods.

Step 1: Choose the key infrastructure. Here schools​ are selected as the critical infrastructure of interest.

Step 2: Use QGIS/ArcGIS tool (such as Zonal Statistics) to figure out maximum inundation depth (denoted as H) within school (or buffer) areas​.

​Step 3: Calculate economic loss of each school area in terms of the economic loss curve. For example F=11616.4*lgH+4030.7* (lgH)2+969.4 *(lgH)3 +30139.1​

​Step 4: Calculate the total expected economic loss of schools​.